# B.3 Software Development Tools（软件开发工具）

ipython增强了python内建的pdb调试工具，还能让我们的代码运行的更快。

# 1 Interactive Debugger（交互式调试器）

运行例子：

In [3]:
cat ../examples/ipython_bug.py

def works_fine():
    a = 5
    b = 6
    assert(a + b == 11)

def throws_an_exception():
    a = 5
    b = 6
    assert(a + b == 10)

def calling_things():
    works_fine()
    throws_an_exception()

calling_things()


In [1]:
run ../examples/ipython_bug.py

AssertionError: 

下面输入`%debug`后能进入调试器，可以输入u(up)和d(down)来查看每一步动作。默认的开始位置是出错最根本的代码段，一直输入u的话，可以一步步回到最开始出错的语句：

In [2]:
%debug

> /Users/xu/DLearning/GithubRepo/pydata-book/examples/ipython_bug.py(9)throws_an_exception()
      7     a = 5
      8     b = 6
----> 9     assert(a + b == 10)
     10 
     11 def calling_things():

ipdb> u
> /Users/xu/DLearning/GithubRepo/pydata-book/examples/ipython_bug.py(13)calling_things()
     11 def calling_things():
     12     works_fine()
---> 13     throws_an_exception()
     14 
     15 calling_things()

ipdb> u
> /Users/xu/DLearning/GithubRepo/pydata-book/examples/ipython_bug.py(15)<module>()
     11 def calling_things():
     12     works_fine()
     13     throws_an_exception()
     14 
---> 15 calling_things()

ipdb> u
> /Users/xu/anaconda/envs/py35/lib/python3.5/site-packages/IPython/utils/py3compat.py(186)execfile()
    184         with open(fname, 'rb') as f:
    185             compiler = compiler or compile
--> 186             exec(compiler(f.read(), fname, 'exec'), glob, loc)
    187 
    188     # Refactor print statements in doctests.

ipdb> u
*** Oldest frame

使用`%pdb`命令能让ipython自动在每一次异常后自动进入调试状态。

假设我们想要在执行一个函数或脚本的时候，设置断点或步骤（breakpoints or step），方便我们检查每一步的状态。可以使用`%run`配合`-d`。


> 译者：要想正常运行，需要重新启动一下Notebook里的Kernel。我怀疑是因为上面运行过了%debug，所以下面无法正常运行。如果是在shell里的话，退出ipython，重新进一下就好了。因为下面的一系列操作在Notebook中是连贯的，为了方便讲解，我会在shell中直接操作，然后截图

使用`%run`配合`-d`，会在执行代码前，先调用调试器。输入s（step）进入脚本：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/usk0k.png)

输入s进入，之后的操作就完全取决于我们自己想要怎么做了。例如，在一个上面的异常中，我们可以在调用works_fine之前先设置一个断点，然后输入c（continue）直到我们碰到断点：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/o0oak.png)

现在，我们可以用s进入works_find()，也可以用n(next)执行works_fine()：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/5hts6.png)

然后，我们可以用s进入throws_an_exception，前进到发生错误的行，并查看范围中的变量。需要注意的是，调试器命令优先于变量名称，这种情况下，使用`!`来查看变量：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/d4pxj.png)

下图是一些调试器的命令：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/gm5pw.png)

**一些利用调试器的其他方法**

有很多方法可以用来调用调试器，第一种是设置一个特殊的set_trace函数（pdb.set_trace），这种情况下是一个Poor Man's Breakpoint。这里有两个小片段大家可能会觉得有用（通常放在ipython的profile中，作者是这么做的）：


    from IPython.core.debugger import Pdb 
    import sys

    def set_trace():
        Pdb(color_scheme='Linux').set_trace(sys._getframe().f_back) 

    def debug(f, *args, **kwargs):
        pdb = Pdb(color_scheme='Linux')
        return pdb.runcall(f, *args, **kwargs)

如果我们想要暂时停下，检查代码的话，可以把set_trace放在代码中的任何位置（例如，放在异常出现之前的地方）：

> 这里我把上面的代码写到了新的文件ipython_bug2.py

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/j6z4b.png)

按c（continue）会让代码继续执行。

我们上面说的debug函数能让我们以一种交互式的方式进行调试。假设我们写了一个函数f，我们希望一步步查看其中的逻辑：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/n6tzf.png)

通常使用f的话像是这样，f(1, 2, z=3)。我们把f作为debug的第一个参数，后面跟着传入f的值：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/0ss2a.png)

这两个函数能节省我们很多时间。

我们上面也用过，调试器可以和%run一起使用，在%run后面加`-d`，就能直接进入调试器：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/jy2jh.png)

在`-b`后面添加行数，能让调试器在有断点的前提下启动：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/gdly7.png)


# 2 Timing Code: %time and %timeit（时间代码%time和%timeit）

通常我们可以使用python自带的time来计算程序的运行时间：
```
import time 
start = time.time() 
for i in range(iterations):
    # some code to run here 
    elapsed_per = (time.time() - start) / iterations
```
不过这种写法比较繁琐。ipython有魔法函数方便我们计算时间，`%time`和`%timeit`。

`%time`会将程序运行一次，然后返回总时间。假设我们有一个很大的保存有string的list，有两个方法用于选出有固定前缀的string，我们想要比较这两个方法的用时。下面的list有600000个string，我们要找的前缀是`foo`：

In [2]:
strings = ['foo', 'foobar', 'baz', 'qux', 'python', 'Guido Van Rossum'] * 100000
method1 = [x for x in strings if x.startswith('foo')]
method2 = [x for x in strings if x[:3] == 'foo']

我们可以用%time查看用时：

In [3]:
%time method1 = [x for x in strings if x.startswith('foo')]

CPU times: user 143 ms, sys: 5.26 ms, total: 148 ms
Wall time: 157 ms


In [4]:
%time method2 = [x for x in strings if x[:3] == 'foo']

CPU times: user 109 ms, sys: 4.56 ms, total: 114 ms
Wall time: 117 ms


wall time（wall-clock time，执行时间）是我们最关心的。看起来方法1耗时更多一些。用%timeit可以得到更精确的测量，它会把程序运行多次然后计算一个平均时间：

In [5]:
%timeit method1 = [x for x in strings if x.startswith('foo')]

10 loops, best of 3: 170 ms per loop


In [6]:
%timeit method2 = [x for x in strings if x[:3] == 'foo']

10 loops, best of 3: 113 ms per loop


对于一些运行时间较短的程序，用%timeit非常适合：

In [7]:
x = 'foobar'
y = 'foo'

In [8]:
%timeit x.startswith(y)

The slowest run took 9.29 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 274 ns per loop


In [9]:
%timeit x[:3] == y

The slowest run took 13.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 245 ns per loop


# 3 Basic Profiling: %prun and %run -p（基本监测：%prun和%run -p）

监测代码和对代码计时很相关，不过前者可以查看时间花在了哪里。python主要的检测工具是cProfile模块，这个模块和ipython毫无关系。cProfile可以监测每一个函数分别用时多少。

一个常见的用法是在命令行使用cProfile，运行整个程序，然后输出每个函数的时间。假设我们想在一个循环里做一些线性代数的计算（计算100×100矩阵的最大特征值）

```
import numpy as np 
from numpy.linalg import eigvals

def run_experiment(niter=100):
    K = 100 
    results = [] 
    for _ in range(niter):
        mat = np.random.randn(K, K) 
        max_eigenvalue = np.abs(eigvals(mat)).max()
        results.append(max_eigenvalue) 
    return results 
some_results = run_experiment() 
print('Largest one we saw: %s' % np.max(some_results))

```

把上面的代码写到一个文件里运行，可以看到每一个函数都会给出运行的时间：
![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/j6c9m.png)

不过输出的内容实在太多，很难找到哪些函数时最花时间的，我们可以使用-s来对结果排序：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/0yr57.png)

如果一个函数中有第二个函数，计时器会把第二个函数的运行时间也算到第一个函数里。

除了命令行中的使用，在不开启一个新线程的情况下，cProfile还能用于检测一段代码的运行。而ipython提供了更方便的魔法函数，%prun。比如我们在ipython环境下输入了上面的代码，然后直接计算run_experiment的运行时间：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/7clqe.png)




在jupyter中我们可以使用%%prun来监测这个代码块的运行。

# 4 Profiling a Function Line by Line（对函数进行每一行的监测）

有时候%prun无法告诉我们一个函数内具体的运行时间。而魔法函数%lprun能做到每一行的监测。我们可以在ipython的设置文件中，打开这个功能：
```
# A list of dotted module names of IPython extensions to load. 
c.TerminalIPythonApp.extensions = ['line_profiler']
```
我们可以运行命令：
```
%load_ext line_profiler
```
line_profiler最强大的地方在于可以和ipython进行交互，假设我们有文件prof_mod：
```
from numpy.random import randn 

def add_and_sum(x, y):
    added = x + y 
    summed = added.sum(axis=1) 
    return summed

def call_function():
    x = randn(1000, 1000) 
    y = randn(1000, 1000) 
    return add_and_sum(x, y)

```

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/y1lgo.png)

如果我们激活ipython中的line_profiler的话，就可以使用%lprun了。这个魔法函数必须要指定对哪一个函数进行监测，下面是语法：

    %lprun -f func1 -f func2 statement_to_profile
    
运行后的效果是这样的：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/d0msh.png)

我们可以像调用add_and_sum一样，调用call_function，查看所有代码的具体用时：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/44b0b.png)

作者通常使用%prun (cProfile)来监测大程序，%lprun (line_profiler)来监测小程序。